In [1]:
pip install gradio

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['GRADIO_ROOT_PATH'] = f"/{os.environ['JUPYTER_NAME']}/proxy/7860"

In [3]:
try:
    demo.close()
except NameError:
    pass

import gradio as gr
from openai import OpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer
import modelscope
from modelscope import AutoModelForCausalLM, AutoTokenizer

2025-01-19 22:04:02.385162: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 22:04:02.423251: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 22:04:03.215762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def system_message():
    return "假设你是一个前端开发资深工程师，擅长对源码进行组件化。"


def generate_prompts(file):
    if not file:
        return "未上传文件", "", ""

    print(type(file))
    file_content = file.content.decode("utf-8")

    description = """
    请将网页源码中的组件分离出来，要求输出封装好的，即插即用的js和css代码，能够让我用id为'component'的组件中直接调用。
    """

    with open('prompt_files/demo0.html', 'r', encoding='utf-8') as file:
        demo_init = file.read()
    with open('prompt_files/demo0.js', 'r', encoding='utf-8') as file:
        demo_js = file.read()
    with open('prompt_files/demo0.css', 'r', encoding='utf-8') as file:
        demo_css = file.read()

    demo_prompt = f"例如：\n源码为：\n {demo_init} \n你应当组件化的js代码为：\n {demo_js} \n你应当组件化的css代码为：\n {demo_css} \n"

    request = f"\n你需要转换的源码如下\n {file_content} \n"
    return description + demo_prompt + request

In [5]:
MODEL_NAME = "Qwen/Qwen2.5-Coder-32B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

2025-01-19 22:04:05,970 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-32B-Instruct].
2025-01-19 22:04:05,970 - modelscope - WARNING - Failed to create symbolic link /mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-32B-Instruct for /mnt/workspace/.cache/modelscope/Qwen/Qwen2___5-Coder-32B-Instruct.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


2025-01-19 22:05:07,545 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-32B-Instruct].
2025-01-19 22:05:07,546 - modelscope - WARNING - Failed to create symbolic link /mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-32B-Instruct for /mnt/workspace/.cache/modelscope/Qwen/Qwen2___5-Coder-32B-Instruct.


In [6]:
# 处理上传文件并生成代码
def generate_code(image, file):
    # 构造 LLM 输入
    messages = [
        {"role": "system", "content": system_message()},
        {"role": "user", "content": generate_prompts(file)}
    ]

    # 生成 组件化 代码
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # TODO: 拆分成js + css
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return result[0]['generated_text'], result[0]['generated_text']

In [7]:
# 构建 Gradio 界面
with gr.Blocks(css=".file-input { height: 30px !important; }") as demo:
    gr.Markdown("<center><h1>组件复制门</h1></center>")
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 输入您想提取的组件截图和源码")
            image_input = gr.Image(label="上传图片")
            file_input = gr.File(label="上传文件", interactive=True, container=False,
                                 elem_classes="file-input")
            submit_btn = gr.Button("提交")

        with gr.Column(scale=1):
            gr.Markdown("### 封装组件表现效果")
            # python_output = gr.Textbox(label="Python 运行结果", interactive=False)

    with gr.Row():
        with gr.Column():
            gr.Markdown("### 封装的 JS 代码")
            generated_ans_js = gr.Code(label="JavaScript 代码", language="javascript")

        with gr.Column():
            gr.Markdown("### 封装的 CSS 代码")
            generated_ans_css = gr.Code(label="CSS 代码", language="css")

    # 绑定交互逻辑
    submit_btn.click(generate_code, inputs=[image_input, file_input],
                     outputs=[generated_ans_js, generated_ans_css])

# 运行 Gradio 应用
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 2047, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2505, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", 